Two-Phase Optimization Strategy and Reliability-Based Optimization Method Using Monte Carlo Simulation
1. Two-Phase Optimization Strategy
1.1 Theoretical Foundation
The two-phase optimization strategy is an effective method for solving reliability-constrained optimization problems, particularly suitable for cases where constraint functions exhibit highly nonlinear characteristics. This method divides the optimization process into two consecutive phases: first finding an approximate feasible solution, then performing precise optimization, which can significantly improve algorithm convergence and robustness.
In reliability optimization problems, the objective function and constraints can be expressed as:
$$ \begin{aligned} \min_{\mathbf{d}} \quad & f(\mathbf{d}) \ \text{s.t.} \quad & \beta_i(\mathbf{d}) \geq \beta_t, \quad i = 1,2,...,m \end{aligned} $$
Where $\mathbf{d}$ is the design variable vector, $f(\mathbf{d})$ is the objective function, $\beta_i(\mathbf{d})$ is the reliability index of the $i$-th constraint, and $\beta_t$ is the target reliability index.
1.2 Algorithm Flow
The core idea of the two-phase optimization strategy is to decompose the difficult reliability-constrained optimization problem into two relatively simpler subproblems:
Phase One: Penalty Function Optimization
In the first phase, the constrained problem is transformed into an unconstrained problem by introducing penalty terms:
$$ \min_{\mathbf{d}} \quad f_p(\mathbf{d}) = f(\mathbf{d}) + \lambda \sum_{i=1}^{m} \max(0, -g_i(\mathbf{d}))^2 $$
Where $\lambda$ is the penalty factor (typically set to a large value to ensure constraint priority over the objective function), and $g_i(\mathbf{d})$ is the constraint function. In the code, conservative estimates are calculated using Kriging model predictions considering prediction errors:
$$ g_i(\mathbf{d}) = \mu_i(\mathbf{d}) - \alpha \cdot \sigma_i(\mathbf{d}) $$
Here, $\mu_i(\mathbf{d})$ is the predicted mean from the Kriging model, $\sigma_i(\mathbf{d})$ is the predicted standard deviation, and $\alpha$ is the safety margin coefficient (set to 1.5 in the code).
The first phase uses the COBYLA algorithm to obtain an intermediate point $\mathbf{d}^{int}$ close to the feasible region.
Phase Two: Explicit Constraint Optimization
In the second phase, starting from the first phase results, explicit constraints are used for precise optimization:
$$ \begin{aligned} \min_{\mathbf{d}} \quad & f(\mathbf{d}) \ \text{s.t.} \quad & \beta_i(\mathbf{d}) - \beta_t - \varepsilon \geq 0, \quad i = 1,2,...,m \end{aligned} $$
Where $\varepsilon$ is a small positive number for numerical stability.
Depending on the use_mcs parameter, there are two ways to calculate reliability constraints:
1. AMV-based constraints: Using the Advanced Mean Value method to calculate reliability indices
2. MCS-based constraints: Using Monte Carlo Simulation to directly calculate reliability indices
The second phase optimization uses the SLSQP algorithm, which is suitable for handling nonlinear constrained optimization problems.
1.3 Convergence Handling and Correction Strategies
When optimization results don't meet constraint requirements, the code implements the following correction strategies:
1. Fallback strategy: Finding feasible solutions between design points and intermediate points $$\mathbf{d}^{new} = \alpha \mathbf{d} + (1-\alpha) \mathbf{d}^{int}, \quad 0 \leq \alpha \leq 1$$
2. Binary search: Gradually adjusting the $\alpha$ value through binary search, with a maximum of 10 attempts to find feasible solutions that satisfy constraints
2. Reliability Optimization Strategy Based on Monte Carlo Simulation
2.1 Theoretical Foundation
Traditional reliability analysis methods (like FORM, AMV) are computationally efficient but have limited accuracy for highly nonlinear problems. Monte Carlo Simulation, while computationally expensive, provides more accurate reliability assessments. The MCS-based optimization strategy introduces MCS into the optimization process to improve the accuracy of reliability constraint calculations.
In this method, reliability constraints are calculated as:
$$ \beta_i = -\Phi^{-1}(P_{f,i}) $$
Where $P_{f,i}$ is the failure probability of the $i$-th constraint calculated through MCS:
$$ P_{f,i} = \frac{1}{N}\sum_{j=1}^{N}I[g_i(\mathbf{x}_j) < 0] $$
Where $N$ is the number of MCS samples, $\mathbf{x}_j$ is a random variable sample, and $I[\cdot]$ is an indicator function that equals 1 when the condition in brackets is satisfied and 0 otherwise.
2.2 MCS-Guided Optimization Algorithm Flow
The main flow of the MCS-guided optimization strategy includes:
1. Initial evaluation: Using MCS to evaluate reliability indices of initial design points
2. Identification of unsatisfied constraints: Determining which constraints don't meet MCS reliability requirements
3. Constraint construction: Creating MCS-based optimization constraints for unsatisfied constraints, adding safety margins for robustness $$\beta_i(\mathbf{d}) - (\beta_t + \text{margin}) \geq 0$$
4. Optimization solution: Using COBYLA optimizer to solve constrained optimization problems
5. Reliability verification: Performing MCS reliability evaluation verification on optimization results
2.3 Alternative Strategy and Differential Evolution
When standard optimization methods fail to find solutions that meet reliability requirements, the code implements an alternative optimization strategy:
1. Reliability-prioritized objective function: Constructing a new objective function dominated by reliability $$\min_{\mathbf{d}} \quad -\min_i(\beta_i(\mathbf{d}) - \beta_t) + \omega \cdot f(\mathbf{d})$$ Where $\omega$ is the weight for objective function penalty (set to 0.1 in the code)
2. Differential evolution algorithm: Using differential evolution (a global optimization method) to solve this objective function and find design points that meet reliability requirements
2.4 Comprehensive Optimization Framework
The MCS-based optimization strategy combined with the two-phase optimization strategy forms a comprehensive optimization framework:
1. First using the two-phase optimization strategy to obtain preliminary optimization results
2. Then using MCS to verify reliability indices
3. If reliability requirements aren't met, initiating MCS-guided optimization for further improvement
4. If needed, using differential evolution algorithm as a final alternative
3. Robust Convergence Judgment
Both optimization strategies employ a robust convergence judgment mechanism considering three aspects:
1. Design point convergence: $$|\mathbf{d}^{new} - \mathbf{d}^{old}| < \varepsilon_d$$
2. Objective function convergence: $$\frac{|f(\mathbf{d}^{new}) - f(\mathbf{d}^{old})|}{|f(\mathbf{d}^{old})|} < \varepsilon_f$$
3. Reliability index convergence: For all constraints $i$, $$\frac{|\beta_i^{new} - \beta_i^{old}|}{|\beta_i^{old}|} < \varepsilon_{\beta}$$
The optimization process only terminates when these three conditions are simultaneously satisfied and all constraints' reliability indices exceed the target reliability index.
4. Summary
The combination of the two-phase optimization strategy and MCS-based reliability optimization method forms an efficient and robust optimization framework that can effectively handle nonlinear reliability-constrained optimization problems. This method significantly improves the ability to solve complex engineering problems through its two-phase approach of first finding approximate feasible solutions then performing precise optimization, combined with the accurate reliability assessments provided by MCS.


```mermaid
flowchart TD
    A[Start] --> B[Initialize parameters\nDesign points, boundary conditions, target reliability indices]
    B --> C[Generate initial sample points\nUsing optimal Latin hypercube design]
    C --> D[Build initial Kriging model]
    D --> E[Begin main iteration loop]
    
    E --> F[MPP calculation\nUsing improved AMV method]
    F --> G[Feasibility check\nEvaluate constraint status]
    G --> H[Select new sample points\nBased on EF function and space-filling criteria]
    H --> I{New sample points\nadded?}
    
    I -->|Yes| J[Update training set\nCalculate constraint values for new points]
    J --> K[Refit Kriging model]
    K --> L[Two-phase optimization\nFind improved design points]
    
    I -->|No| L
    
    L --> M[Evaluate convergence\nDesign points, objective function, reliability indices]
    M --> N{Converged?}
    
    N -->|No| O{Reached maximum\niterations?}
    O -->|Yes| P[Output current optimal results]
    O -->|No| E
    
    N -->|Yes but reliability\nrequirements not met| Q[MCS-guided optimization]
    Q --> R{MCS optimization\nfound feasible solution?}
    R -->|Yes| S[Update with MCS results]
    R -->|No| T[Keep current design]
    
    S --> P
    T --> P
    
    N -->|Yes and reliability\nrequirements met| P
    
    P --> U[Perform final MCS verification]
    U --> V{Meet reliability\nrequirements?}
    
    V -->|No| W[Final MCS-guided optimization]
    W --> X{Found feasible\nsolution?}
    X -->|Yes| Y[Update with final feasible design]
    X -->|No| Z[Keep current results]
    
    V -->|Yes| Y
    
    Y --> AA[Visualize results\nPlot optimization history and final constraints]
    Z --> AA
    
    AA --> BB[End]
    
    subgraph Two-Phase Optimization
    L1[Phase 1: Use penalty function\nto find points near feasible region]
    L2[Evaluate if design points\nmeet constraints]
    L3[Phase 2: Explicit constraint\noptimization based on MCS or AMV]
    L4[Verify if final design\npoints meet constraints]
    L5[If not, execute fallback\nstrategy to find feasible solutions]
    
    L1 --> L2 --> L3 --> L4
    L4 -->|Not met| L5
    end
    
    subgraph MCS-Guided Optimization
    Q1[Create optimization problem\nwith MCS reliability constraints]
    Q2[Solve using COBYLA optimizer]
    Q3[Evaluate MCS reliability\nof optimization results]
    Q4{Any improvement?}
    Q5[Try alternative using\ndifferential evolution]
    Q6[Evaluate alternative]
    
    Q1 --> Q2 --> Q3 --> Q4
    Q4 -->|No| Q5 --> Q6
    end
```